# <span style="color: blue;">Исключения</span>

### Зачем нужны исключения?

Исключения нужны для **исключительных ситуаций**, например:

не удалось выделить память для объекта

In [ ]:
[0] * int(1e9)

импортируемый модуль не был найден

In [ ]:
import foobar

программист написал код, складывающий список и число

In [ ]:
 [1, 2, 3] + 4

**Исключения** — это ошибки, которые можно обрабатывать.

### Обработка исключений: `try`...`except`

Для обработки исключений в Python используются операторы `try` и `except`:

In [ ]:
try:
    something_dangerous()
except (ValueError, ArithmeticError):
    pass
except TypeError as e:
    pass

Ветка `except` принимает два аргумента:
1. выражение, возвращающее тип или кортеж типов,
2. опциональное имя для перехваченного исключения.

Исключение **`e`** обрабатывается веткой **`except`**, если её первый аргумент **`expr`** можно сопоставить с исключением: `isinstance(`**`e`**`,` **`expr`**`)`

При наличии нескольких веток `except` интерпретатор сверху вниз ищет подходящую.

Если никакая ветка не сработает, то выбросим исключение наверх

С помощью **`as`** исключение можно связать с переменной (это может быть полезно)

### Особенности для блока `except`

На месте выражения в ветке `except` может стоять любое выражение, например, вызов функции или обращение к переменной:

In [ ]:
try:
    something_dangerous()
except Exception as e:
    try:
        something_else()
    except type(e): # Какое исключение мы
        pass # перехватим?

Время жизни переменной `e` ограничивается веткой `except`:

In [ ]:
try:
    1 + "42"
except TypeError as e:
    pass # Что делать, если нам нужно e?

e

Если была какая-то переменная с таким именеме до этого -- она уничтожится

In [ ]:
e = 'hello'

try:
    1 + "42"
except TypeError as e:
    pass

e

### Встроенные исключения

`BaseException` — базовый класс для встроенных исключений в Python.

In [2]:
BaseException.__subclasses__()

[Exception, GeneratorExit, SystemExit, KeyboardInterrupt]

Напрямую от класса `BaseException` наследуются:
* системные исключения, приводящие к завершению работы интерпретатора _(как правило, их не надо обрабатывать)_
    * `SystemExit` вызывается, когда мы вызываем `sys.exit()`
    * `KeyboardInterrupt` вызывается когда нажимаем `Ctrl+C`<br/><br/>
* по поводу `GeneratorExit` поговорим позже

Все остальные встроенные исключения _(и исключения, объявленные пользователем)_ должны наследоваться от **`Exception`**.

Поэтому чтобы обработать любое исключение, достаточно написать:

In [ ]:
try:
    something_dangerous()
except Exception: # Почему не BaseException?
    pass

### `AssertionError`

Исключение `AssertionError` поднимается, когда условие оператора `assert` не выполняется

In [ ]:
assert 2 + 2 == 5, ("Math", "still", "works")

Оператор `assert` используется для ошибок, которые могут возникнуть только в результате ошибки программиста.

Поэтому перехватывать `AssertionError` считается дурным тоном.

### `ImportError` и `NameError`

Если оператор **`import`** не смог найти модуль с указанным именем, поднимается исключение **`ImportError`**:

In [ ]:
import foobar

В Python 3.6 появился новый класс-наследник для этого: **`ModuleNotFoundError`**

**`NameError`** поднимается, если не была найдена локальная или глобальная переменная:

In [ ]:
foobar

### `AttributeError` и `LookupError`

Исключение `AttributeError` поднимается при попытке прочитать/удалить или (в случае `__slots__`) записать значение в
несуществующий атрибут:

In [ ]:
object().foobar

Исключения `KeyError` и `IndexError` наследуются от базового класса `LookupError` и поднимаются, если в контейнере нет элемента по указанному ключу или индексу:

In [ ]:
{}["foobar"]

In [ ]:
[][0]

### `ValueError` и `TypeError`

Исключение `ValueError` используется в случаях, когда другие более информативные исключения, например, `KeyError`, не применимы:

In [ ]:
"foobar".split("")

Исключение `TypeError` поднимается, когда оператор/функция/метод вызываются с аргументом несоответствующего типа:

In [ ]:
b"foo" + "bar"

Полный список исключений можно найти в документации языка:
* https://docs.python.org/3/library/exceptions.html

### Исключения, объявленные пользователем

Для объявления нового типа исключения достаточно объявить класс, наследующийся от базового класса `Exception`.

Хорошая практика при написании библиотек на Python — объявлять свой базовый класс исключений, например:

In [20]:
class SlidesException(Exception):
    pass

In [21]:
class TestFailure(SlidesException):
    def __str__(self):
        return "lecture test failed"

Наличие базового класса позволяет пользователю обработать любое исключение, специфичное для библиотеки в одной ветке `except`:

In [ ]:
try:
    do_something()
except SlidesException:
    pass  # ...

### Интерфейс исключений

Интерфейс исключений в Python довольно нехитрый:
* атрибут **`args`** — кортеж аргументов, переданных конструктору исключения
* атрибут **`__traceback__`** — содержит информацию о стеке вызовов на момент возникновения исключения

In [24]:
try:
    1 + "42"
except Exception as e:
    caught = e

caught.args

("unsupported operand type(s) for +: 'int' and 'str'",)

In [25]:
caught.__traceback__

In [27]:
import traceback

traceback.print_tb(caught.__traceback__)

  File "<ipython-input-24-be36e5c76990>", line 2, in <module>
    1 + "42"


### Оператор `raise`

Поднять исключение можно с помощью оператора **`raise`** _(поднимаем исключение по стеку наверх)_:

In [ ]:
raise TypeError("type mismatch")

Если не нужно никакое пояснение к ошибке, то можно поднимать класс исключения, а не объект:

In [ ]:
raise TypeError

Аргумент оператора **`raise`** должен наследоваться от базового класса **`BaseException`**:

In [ ]:
raise 42

Если вызвать оператор **`raise`** без аргумента, то он поднимет последнее пойманное исключение:

In [ ]:
try:
    1 / 0
except Exception as e:
    raise e

In [ ]:
try:
    1 / 0
except Exception:
    raise

Если последнего активного исключения нет, то **`raise`** поднимет **`RuntimeError`**.

In [ ]:
raise

### Оператор `raise from`

In [ ]:
try:
    {}["foobar"]
except KeyError as e:
    raise RuntimeError("Ooops!") from e

Бывает полезно, когда исключение возникает в результате другого исключения

Связанное исключение хранится в атрибуте **`__cause__`**

### Операторы: `try`...`finally`

Иногда требуется выполнить какое-то действие вне зависимости от того, произошло исключение или нет, например, закрыть файл:

In [ ]:
try:
    handle = open("example.txt", "wt")
    try:
        do_something(handle)
    finally:
        handle.close()
except IOError as e:
    print(e, file=sys.stderr)

Аналогичным образом нужно работать с любыми другими ресурсами: сетевыми соединениями, примитивами синхронизации и т.п.

### Операторы: `try`...`else`

С помощью ветки `else` можно выполнить какое-то действие в ситуации, когда внутри `try` блока не возникло исключения:

In [ ]:
try:
    handle = open("example.txt", "wt")
except IOError as e:
    print(e, file=sys.stderr)
else:
    report_success(handle)

Чем использование `else` лучше следующего варианта?

In [ ]:
try:
    handle = open("example.txt", "wt")
    report_success(handle)
except IOError as e:
    print(e, file=sys.stderr)

### Цепочки исключений: `except` и оператор `raise`

In [ ]:
try:
    {}["foobar"]
except KeyError:
    "foobar".split("")

### Цепочки исключений: `finally` и оператор `raise`

In [ ]:
try:
    {}["foobar"]
finally:
    "foobar".split("")

### Исключения: резюме

Механизм обработки исключений в Python похож на аналогичные конструкции в С++ и Java. 

Но Python расширяет привычные `try, except, finally` дополнительной веткой **`else`**.

Поднять исключение можно с помощью оператора **`raise`**, его семантика эквивалентна `throw` в C++ и Java.

В Python много **встроенных типов исключений**, которые можно и нужно использовать при написании функций и методов.

Для **объявления нового типа исключения** достаточно унаследоваться от базового класса **`Exception`**.

Два важных правила при работе с исключениями:
* минимизируйте размер ветки `try`
* всегда старайтесь использовать наиболее **специфичный** тип исключения в ветке `except`